In [140]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

### 1  Initializing Model

In [141]:
m = pyo.ConcreteModel()

### 2  Set Parameteres (What is given?)

In [142]:
# different projects to choose from
m.setPg = pyo.Set(initialize=['A', 'B', 'C', 'D', 'E'])
m.setPd = pyo.Set(initialize=['X', 'Y', 'Z'])  # number of working days
m.g = {
    'A': 20,
    'B': 10,
    'C': 40,
    'D': 50,
    'E': 5
}  #revenue by project
m.d = {
    'X': 50,
    'Y': 20,
    'Z': 30,
}  #revenue by project
m.C = {
    'A': 0.10,
    'B': 0.05,
    'C': 0.30,
    'D': 0.40,
    'E': 0.01
}  #revenue by project
m.dsum = sum(m.d[d_id] for d_id in m.d)

In [143]:
m.g.keys()

dict_keys(['A', 'B', 'C', 'D', 'E'])

###   Variables (What do we need to find)?

In [144]:
m.Pg = pyo.Var(m.setPg, bounds=(0,None))
Pg = m.Pg

In [145]:
Pg['A']

### Objective Function (How do we find the best possible outcome)?¶

### Constrains (What are the conditions we need to keep in mind)?

- total power generated == total power consumed by the load point
- power generated for a g is greater than 0
- power generated at each g is less than equal to its limit
- power consumed at load point 0 is less than the sum of power generated at 0 and 3

In [146]:
pg_sum = sum([Pg[g_id] for g_id in m.g])

In [147]:
m.balance = pyo.Constraint(expr = pg_sum == m.dsum)

In [148]:
m.cond = pyo.Constraint(expr = Pg['A']+Pg['D'] >= m.d['X'])

In [149]:
m.produced = pyo.ConstraintList()
for power_station in m.setPg:
    m.produced.add(expr = Pg[power_station]>=0)

In [150]:
m.Limit = pyo.ConstraintList()
for p_station in m.setPg:
    m.Limit.add(expr = Pg[p_station] <= m.g[p_station])

### Objective Function (How do we find the best possible outcome)?

In [151]:
cost_sum = sum([Pg[power_station]*m.C[power_station] for power_station in m.setPg])
m.obj = pyo.Objective(expr = cost_sum, sense=pyo.minimize)

### Solveing using Solver

In [152]:
opt = SolverFactory('gurobi')
results = opt.solve(m)

### Results

In [153]:
[pyo.value(Pg[g]) for g in m.setPg]

[20.0, 10.0, 35.0, 30.0, 5.0]

In [154]:
m.pprint()

4 Set Declarations
    Limit_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    produced_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    setPd : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'X', 'Y', 'Z'}
    setPg : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'A', 'B', 'C', 'D', 'E'}

1 Var Declarations
    Pg : Size=5, Index=setPg
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          A :     0 :  20.0 :  None : False : False :  Reals
          B :     0 :  10.0 :  None : False : False :  Reals
          C :     0 :  35.0 :  None : False : False :  Reals
          D :     0 :  30.0 :  None : False : False :  

In [155]:
m.Pg.pprint()

Pg : Size=5, Index=setPg
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      A :     0 :  20.0 :  None : False : False :  Reals
      B :     0 :  10.0 :  None : False : False :  Reals
      C :     0 :  35.0 :  None : False : False :  Reals
      D :     0 :  30.0 :  None : False : False :  Reals
      E :     0 :   5.0 :  None : False : False :  Reals


### Summary

In [156]:
print(results)


Problem: 
- Name: x6
  Lower bound: 25.05
  Upper bound: 25.05
  Number of objectives: 1
  Number of constraints: 13
  Number of variables: 6
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 6
  Number of nonzeros: 18
  Sense: minimize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 0.0005481243133544922
  Error rc: 0
  Time: 0.09047603607177734
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

